In [46]:
import numpy as np
import pandas
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

In [60]:
# import data from csv
dataframe = pandas.read_csv('heart.csv',usecols = range(0,14), engine='python')
dataset = dataframe.values

In [61]:
# shuffle dataset
np.random.shuffle(dataset)

# divide data (X) and targets (Y)
X = np.delete(dataset, obj=13, axis=1)
Y = dataset[:,13]

# normalize data (X)
X = sklearn.preprocessing.normalize(X, norm='l1', axis=0)


In [73]:
# method to calculate accuracy
def calcAccuracy(actual, predicted):
#     correct = 0
#     for i in range(0, len(actual)):
#         if actual[i] == predicted[i]:
#             correct += 1
#         print "Actual: {0} Predicted: {1}".format(actual[i], predicted[i])
#     return precision_score(actual, predicted)
#     return confusion_matrix(actual, predicted)
    return accuracy_score(actual, predicted, normalize = True)


# method to test train/test split by percentage
def splitByPercentageValidation(train_size=0.7):
    X_train, X_test, Y_train, Y_test = \
        train_test_split(X, Y, train_size = train_size, random_state=2017)

    # run on test and train data
    clf = buildClf(X_train, Y_train)
    
    predicted = clf.predict(X_test)    
    test_accuracy = calcAccuracy(Y_test, predicted)
    
    predicted = clf.predict(X_train)
    train_accuracy = calcAccuracy(Y_train, predicted)
    
    return train_accuracy, test_accuracy


# method for k-fold cross validation
def kFoldValidation(n_splits=10):
    kFold = KFold(n_splits=n_splits)

    # run on test data
    test_results = []
    train_results = []
    for train_index, test_index in kFold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        # build classifier for each set
        clf = buildClf(X_train, Y_train)
        
        predicted = clf.predict(X_test)
        test_accuracy = calcAccuracy(Y_test, predicted)
        
        predicted = clf.predict(X_train)
        train_accuracy = calcAccuracy(Y_train, predicted)
        
        test_results.append(test_accuracy)
        train_results.append(train_accuracy)
#     return train_accuracy, test_accuracy
    return np.mean(train_results), np.mean(test_results)


# method for leave one out cross validation
def looValidation():
    loo = sklearn.model_selection.LeaveOneOut()

    # run on test data
    test_results = []
    train_results = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        # build classifier for each set
        clf = buildClf(X_train, Y_train)
        
        predicted = clf.predict(X_test)
        test_accuracy = calcAccuracy(Y_test, predicted)
        predicted = clf.predict(X_train)
        train_accuracy = calcAccuracy(Y_train, predicted)
        
        test_results.append(test_accuracy)
        train_results.append(train_accuracy)
    
#     return train_accuracy, test_accuracy
    return np.mean(train_results), np.mean(test_results)
    

In [77]:
# method to build classifier
def buildClf(train_data, train_target):
#     clf = DecisionTreeClassifier()
#     clf = GaussianNB()
#     clf = LogisticRegression()
#     clf = SVC()
#     clf = RandomForestClassifier()
#     clf = KNeighborsClassifier(6)
#     clf = MultinomialNB()
#     clf = BernoulliNB()
#     clf = GaussianProcessClassifier()
#     clf = AdaBoostClassifier(n_estimators =30)
#     clf = GradientBoostingClassifier(n_estimators = 100)
#     clf = QuadraticDiscriminantAnalysis()
#     clf = MLPClassifier()
#     clf = BaggingClassifier(QuadraticDiscriminantAnalysis(), max_samples=0.2, max_features=0.2)
    clf = BaggingClassifier(GaussianNB(), max_samples=0.2, max_features=0.2)
    clf.fit(train_data, train_target)
    return clf

In [78]:
splitByPercentage_train, splitByPercentage_test = splitByPercentageValidation(0.85)
kFold_train, kFold_test = kFoldValidation(10)
loo_train, loo_test = looValidation()

print "-----Train-----"
print "Split by % Accuracy: {}%".format(splitByPercentage_train*100)
print "K-Fold Accuracy: {}%".format(kFold_train*100)
print "Leave-One-Out Accuracy: {}%".format(loo_train*100)

print "-----Test-----"
print "Split by % Accuracy: {}%".format(splitByPercentage_test*100)
print "K-Fold Accuracy: {}%".format(kFold_test*100)
print "Leave-One-Out Accuracy: {}%".format(loo_test*100)

-----Train-----
Split by % Accuracy: 85.5895196507%
K-Fold Accuracy: 85.8847736626%
Leave-One-Out Accuracy: 85.9066501446%
-----Test-----
Split by % Accuracy: 85.3658536585%
K-Fold Accuracy: 83.3333333333%
Leave-One-Out Accuracy: 83.7037037037%


In [ ]:
# visualize the decision tree
# from: http://scikit-learn.org/stable/modules/tree.html

# import pydotplus 
# dot_data = tree.export_graphviz(clf, out_file=None) 
# graph = pydotplus.graph_from_dot_data(dot_data) 
# graph.write_pdf("decision-tree.pdf") 
